In [ ]:
# Execute to get a new list of reach IDs to select
import xarray as xr
import os
import numpy as np

ipath = '/glade/work/mizukami/test_mizuRoute/NHDPlus_testCase/output'
opath = '/glade/work/nijssen/scratch'

id_list_file = os.path.join(opath, 'rout_id.txt')
ifile = 'runoff_1951-1-1.map.gnu.bmrk.nc'
ofile = ifile.replace('.nc', '_subset.nc')

n = 100
ds = xr.open_dataset(os.path.join(ipath, ifile))
np.savetxt(id_list_file, np.random.choice(ds.reachID.values, n, replace=False), fmt="%ld")

In [ ]:
%%time

import xarray as xr
import os

ipath = '/glade/work/mizukami/test_mizuRoute/NHDPlus_testCase/output'
opath = '/glade/work/nijssen/scratch'

id_list_file = os.path.join(opath, 'rout_id.txt')
ifile = 'runoff_1951-1-1.map.gnu.bmrk.nc'
ofile = ifile.replace('.nc', '_subset.nc')

with open(id_list_file) as f:
    ids = [int(x) for x in f]

ds_org = xr.open_dataset(os.path.join(ipath, ifile))
ds = ds_org.drop_dims('hru')

ds_subset = ds.where(ds.reachID.isin(ids), drop=True)

ds_subset = ds_subset.merge(ds_org.drop_dims('seg'))

# make sure that the subsetted types are the same as the original ones
for var in ds_subset.variables:
    ds_subset[var] = ds_subset[var].astype(ds_org[var].dtype)

ds_subset.to_netcdf(os.path.join(opath, ofile))

In [ ]:
a = set(ds_subset.reachID.values)

set(ids).difference(a)